In [1]:
import torch.nn as nn
import torch
import torchaudio
import matplotlib.pyplot as plt

import csv
import numpy as np

In [2]:
from models import ASTModel

In [3]:
from utilities import *

In [4]:
audio_model = ASTModel(label_dim=50, fstride=10, tstride=10, input_fdim=128,
                                  input_tdim=512, imagenet_pretrain=True,
                                  audioset_pretrain=True, model_size='base384')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

path = '/kuacc/users/fsofian19/COMP491_model/models/ssast_istangul/src/finetune/custom/exp/train01-custom-b16-lr1e-4-ft_avgtok-base-1x-noiseTrue-50class/models/audio_model.7.pth'

sd = torch.load(path, map_location=device)

if not isinstance(audio_model, nn.DataParallel):
    audio_model = nn.DataParallel(audio_model)
audio_model = audio_model.to(device)
audio_model.load_state_dict(sd,strict=False)

audio_model.eval();

---------------AST Model Summary---------------
ImageNet pretraining: True, AudioSet pretraining: True
frequncey stride=10, time stride=10
number of patches=600


In [5]:
def get_fbank(filename):
        # mixup

        waveform, sr = torchaudio.load(filename)
        waveform = waveform - waveform.mean()


        fbank = torchaudio.compliance.kaldi.fbank(waveform, htk_compat=True, sample_frequency=sr, use_energy=False,
                                                  window_type='hanning', num_mel_bins=128, dither=0.0, frame_shift=10)

        target_length = 512
        n_frames = fbank.shape[0]

        p = target_length - n_frames

        # cut and pad
        if p > 0:
            m = torch.nn.ZeroPad2d((0, 0, 0, p))
            fbank = m(fbank)
        elif p < 0:
            fbank = fbank[0:target_length, :]


        return fbank

In [34]:
cur_fbank = get_fbank('/kuacc/users/bbiner21/birdclef-2022/birdclef-2022-16khz/skylar/XC362045.ogg')

#/kuacc/users/bbiner21/birdclef-2022/birdclef-2022-16khz/akiapo/XC122401.ogg
#/kuacc/users/bbiner21/ast/egs/custom/data/custom_data/audio_16k/XC109605.ogg
# plt.figure()
# cur_fbank = torch.flip(cur_fbank.t(), [0])
# plt.imshow(cur_fbank.numpy(), cmap='gray',aspect='auto')
print(cur_fbank.shape)

norm_mean = -6.0138397
norm_std = 4.589279

cur_fbank = (cur_fbank - norm_mean) / (norm_std * 2)

cur_fbank = cur_fbank.unsqueeze(0)
out = audio_model(cur_fbank)
out = torch.sigmoid(out)
out

torch.Size([512, 128])


tensor([[0.3119, 0.1915, 0.2662, 0.1010, 0.2327, 0.4213, 0.1695, 0.3143, 0.1613,
         0.1568, 0.3740, 0.3630, 0.2916, 0.4650, 0.3148, 0.3887, 0.2310, 0.2693,
         0.7696, 0.2022, 0.2180]], grad_fn=<SigmoidBackward>)

In [35]:
print(out.shape)

torch.Size([1, 21])


In [36]:
temp = torch.zeros(1,21)
temp[0,18] = 1.

In [31]:
temp

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0.]])

In [37]:
a = calculate_stats(out.detach(),temp)

1
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
